In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter

torch.set_printoptions(linewidth=120) 

In [2]:
train_set=torchvision.datasets.FashionMNIST(
    root="./sample_data/FashionMNIST",
    train=True,
    download=True,
    transform=transforms.Compose([
            transforms.ToTensor()
        ])
    )



Extracting ./sample_data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./sample_data/FashionMNIST/FashionMNIST/raw



Extracting ./sample_data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./sample_data/FashionMNIST/FashionMNIST/raw



Extracting ./sample_data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./sample_data/FashionMNIST/FashionMNIST/raw



Extracting ./sample_data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./sample_data/FashionMNIST/FashionMNIST/raw
Processing...
Done!


In [0]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)

        self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=60)
        self.out=nn.Linear(in_features=60,out_features=10)

    def forward(self,t):

        t=t

        t=F.relu(self.conv1(t))
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        t=F.relu(self.conv2(t))
        t=F.max_pool2d(t,kernel_size=2,stride=2)

        t=F.relu(self.fc1(t.reshape(-1,12*4*4)))
        t=F.relu(self.fc2(t))
        t=self.out(t)

        return t
        

In [4]:
torch.set_grad_enabled(False)

In [0]:
network=Network()

In [6]:
sample=next(iter(train_set))

image,label=sample
image.shape

image.unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

In [0]:
pred=network(image.unsqueeze(0))

In [8]:
pred.shape

torch.Size([1, 10])

In [9]:
pred

tensor([[ 0.0468, -0.1236, -0.0918,  0.0061, -0.0575, -0.0558,  0.0279,  0.0119, -0.1058,  0.0930]])

In [10]:
pred.argmax(dim=1)

tensor([9])

In [11]:
F.softmax(pred,dim=1)

tensor([[0.1072, 0.0904, 0.0933, 0.1029, 0.0966, 0.0967, 0.1052, 0.1035, 0.0920, 0.1122]])

In [12]:
F.softmax(pred,dim=1).sum()

tensor(1.)

In [13]:
F.softmax(pred,dim=1).argmax(dim=1)

tensor([9])

In [0]:
data_loader=torch.utils.data.DataLoader(
    train_set,
    batch_size=10
)

In [15]:
batch=next(iter(data_loader))
images,labels=batch

images.shape

torch.Size([10, 1, 28, 28])

In [0]:
preds=network(images)

In [17]:
preds.shape

torch.Size([10, 10])

In [18]:
preds

tensor([[ 0.0468, -0.1236, -0.0918,  0.0061, -0.0575, -0.0558,  0.0279,  0.0119, -0.1058,  0.0930],
        [ 0.0478, -0.1203, -0.0958,  0.0102, -0.0652, -0.0578,  0.0303,  0.0130, -0.1062,  0.0967],
        [ 0.0450, -0.1250, -0.0979,  0.0102, -0.0629, -0.0532,  0.0281,  0.0138, -0.1043,  0.0947],
        [ 0.0438, -0.1255, -0.0966,  0.0143, -0.0615, -0.0532,  0.0290,  0.0143, -0.1065,  0.0954],
        [ 0.0455, -0.1265, -0.0979,  0.0130, -0.0615, -0.0525,  0.0294,  0.0135, -0.1077,  0.0950],
        [ 0.0517, -0.1257, -0.0940,  0.0127, -0.0620, -0.0530,  0.0296,  0.0147, -0.1057,  0.0934],
        [ 0.0488, -0.1273, -0.0915,  0.0102, -0.0603, -0.0586,  0.0319,  0.0152, -0.1050,  0.0965],
        [ 0.0523, -0.1196, -0.0916,  0.0093, -0.0607, -0.0576,  0.0281,  0.0133, -0.1045,  0.0889],
        [ 0.0460, -0.1294, -0.0909,  0.0072, -0.0638, -0.0604,  0.0306,  0.0150, -0.1012,  0.0906],
        [ 0.0404, -0.1321, -0.0924,  0.0104, -0.0643, -0.0570,  0.0297,  0.0121, -0.1006,  0.0911]])

In [19]:
preds.argmax(dim=1)


tensor([9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [20]:
labels

tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])

In [21]:
preds.argmax(dim=1).eq(labels).sum()

tensor(1)

In [0]:
def getNumCorrect(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()
    #lmao


In [23]:
getNumCorrect(preds,labels)

1

In [24]:
#train on a batch
torch.set_grad_enabled(True)
network=Network()

train_loader=torch.utils.data.DataLoader(train_set,batch_size=100)
optimiser=optim.Adam(network.parameters(),lr=0.01)

batch=next(iter(train_loader))
images, labels =batch

preds=network(images)
loss=F.cross_entropy(preds,labels)

loss.backward()
optimiser.step()

print(loss.item())
preds=network(images)
loss=F.cross_entropy(preds,labels)
print(loss.item())

2.303853988647461
2.285881996154785


In [25]:
## training for an entire epoch
torch.set_grad_enabled(True)
network=Network()

train_loader=torch.utils.data.DataLoader(train_set,batch_size=100)
optimiser=optim.Adam(network.parameters(),lr=0.01)
for epoch in range(5):
    totalLoss=0
    totalCorrect=0
    for batch in train_loader:
    # batch=next(iter(train_loader))
        images, labels =batch

        preds=network(images)
        loss=F.cross_entropy(preds,labels)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        totalLoss+=loss.item()
        totalCorrect+=getNumCorrect(preds, labels)

    print("Epoch: ",epoch,"\tTotal Correct: ",totalCorrect,"\tLoss: ", totalLoss)

Epoch:  0 	Total Correct:  47044 	Loss:  340.42355658113956
Epoch:  1 	Total Correct:  51057 	Loss:  240.153990983963
Epoch:  2 	Total Correct:  51943 	Loss:  218.30783687531948
Epoch:  3 	Total Correct:  52232 	Loss:  209.88756828010082
Epoch:  4 	Total Correct:  52490 	Loss:  203.30410832166672


In [26]:
totalCorrect/len(train_set)

0.8748333333333334

In [0]:
#now what we will try to dp is cerate the confusion matrix but for that we require al teh predictions

def getAllPred(model,loader):#we cant pass the whole of the dataset due to memory inefficiency
    allPreds=torch.tensor([])
    for batch in loader:
        images, labels= batch
        preds=model(images)
        allPreds=torch.cat((allPreds,preds),dim =0)

    return allPreds

In [0]:
with torch.no_grad():
    prediction_loader=torch.utils.data.DataLoader(train_set, batch_size=10000)
    train_preds=getAllPred(network,prediction_loader)

In [29]:
predCorrect=getNumCorrect(train_preds,train_set.targets)

print(predCorrect,predCorrect/len(train_set))

52511 0.8751833333333333


In [0]:
p=train_preds.argmax(dim=1)

In [31]:
train_set.targets.shape

torch.Size([60000])

In [32]:
#found out the confusion matrix

cm=torch.zeros(10,10,dtype=torch.int32)
cm
a=train_set.targets
b=train_preds.argmax(dim=1)

print(a,b)
combined=torch.stack((a,b),dim=1)
combined.shape
combined
for x,y in combined:
    cm[x][y]+=1
cm

tensor([9, 0, 0,  ..., 3, 0, 5]) tensor([9, 0, 0,  ..., 3, 0, 5])


tensor([[5562,    6,   52,   69,   11,    1,  246,    0,   53,    0],
        [  25, 5865,    2,   91,    6,    3,    4,    0,    4,    0],
        [ 108,    8, 4660,   79,  796,    4,  330,    0,   15,    0],
        [ 380,   31,   10, 5368,  130,    0,   75,    0,    6,    0],
        [  20,    6,  329,  335, 5170,    2,  123,    0,   15,    0],
        [   0,    0,    1,    2,    0, 5762,    0,  192,   12,   31],
        [1593,    8,  528,  126, 1005,    1, 2679,    0,   59,    1],
        [   0,    0,    0,    0,    0,   26,    0, 5890,    8,   76],
        [  23,    5,   10,   37,   23,    9,   33,    5, 5853,    2],
        [   0,    0,    0,    1,    1,   38,    1,  256,    1, 5702]], dtype=torch.int32)

In [33]:
#stack vs concat

a=torch.tensor([1,1,1,1,1])
b=torch.tensor([2,2,2,2,2])

c=torch.cat((a,b),dim=0)
print(a,b,c)

d=torch.cat((a.unsqueeze(1),b.unsqueeze(1)),dim=1)
print("====================\n",a.unsqueeze(1),"\n",b.unsqueeze(1),"\n",d)

print(d.unsqueeze(2))
print(d.unsqueeze(1))
print(d.unsqueeze(0))

tensor([1, 1, 1, 1, 1]) tensor([2, 2, 2, 2, 2]) tensor([1, 1, 1, 1, 1, 2, 2, 2, 2, 2])
 tensor([[1],
        [1],
        [1],
        [1],
        [1]]) 
 tensor([[2],
        [2],
        [2],
        [2],
        [2]]) 
 tensor([[1, 2],
        [1, 2],
        [1, 2],
        [1, 2],
        [1, 2]])
tensor([[[1],
         [2]],

        [[1],
         [2]],

        [[1],
         [2]],

        [[1],
         [2]],

        [[1],
         [2]]])
tensor([[[1, 2]],

        [[1, 2]],

        [[1, 2]],

        [[1, 2]],

        [[1, 2]]])
tensor([[[1, 2],
         [1, 2],
         [1, 2],
         [1, 2],
         [1, 2]]])


In [35]:
#tensorboard


!tensorboard --version

1.15.0


TensorBoard 1.15.0 at http://2e0bf8f60133:6006/ (Press CTRL+C to quit)


In [39]:
ls

sample_data/
